In [1]:

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

## 데이터 읽어오기.
df = pd.read_csv("../dataset/BostonHousing.csv")





1 Physical GPUs, 1 Logical GPUs


In [2]:


## key말고 해당하는 value값만 추출해서 입력과 정갑 데이터 분류
dataset = df.copy()

X = dataset
Y = dataset.pop('MEDV')


In [3]:

# 모델의 설정
input_Layer = tf.keras.layers.Input((13, ))
x = tf.keras.layers.Dense(10, activation="sigmoid")(input_Layer)
x = tf.keras.layers.Dense(20, activation="sigmoid")(x)
x = tf.keras.layers.Dense(30, activation="sigmoid")(x)
Out_Layer = tf.keras.layers.Dense(1, activation=None)(x)

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                140       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 1,021
Trainable params: 1,021
Non-trainable params: 0
_________________________________________________________________


In [4]:

## 모델 loss, 학습 방법 선언
loss=tf.keras.losses.mean_squared_error
optimizer=tf.keras.optimizers.SGD(learning_rate=0.05)

## 모델 컴파일
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

## 모델 동작
result=model.fit(X, Y, epochs=1000, batch_size=100) # 모델 fit 해서 동작하기

## model fit은 histoy를 반환한다. 훈련중의 발생하는 모든 정보를 담고 있는 딕셔너리.
## histoy는 딕셔너리이므로 keys()를 통해 출력의 key(카테고리)를 알 수 있다.
print(result.history.keys())


SyntaxError: invalid syntax (<ipython-input-4-b4223cb2f775>, line 2)

In [ ]:

### history에서 loss key를 가지는 값들만 추출
loss = result.history['loss']

### loss그래프화
epochs = range(1, len(loss) + 1)
plt.subplot(211)  ## 2x1 개의 그래프 중에 1번째
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### history에서 root_mean_squared_error key를 가지는 값들만 추출
rmse = result.history['root_mean_squared_error']
epochs = range(1, len(rmse) + 1)
### mean_absolute_error를 그래프화
plt.subplot(212)  ## 2x1 개의 그래프 중에 2번째
plt.plot(epochs, rmse, 'ro', label='Training rmse')
plt.title('Training rmse')
plt.xlabel('Epochs')
plt.ylabel('rmse')
plt.legend()

print("\n Test rmse: %.4f" % (model.evaluate(X, Y)[1]))
plt.show()
